# spiral_wasm

In [ ]:
open rust.rust_operators
open rust
open sm'_operators

## spiral_wasm

### get_args

In [ ]:
inl get_args () =
    {
        exception = "exception", 'e'
        trace_level = "trace_level", 't'
        wasm = "wasm", 'w'
    }

### get_command

In [ ]:
let get_command () =
    ##"command"
    |> runtime.new_command
    |> runtime.command_args_override_self true
    |> runtime.command_init_arg (get_args () .exception) (
        runtime.arg_action runtime.SetTrue
    )
    |> runtime.command_init_arg (get_args () .trace_level) (
        real runtime.arg_union `trace_level ignore
    )
    |> runtime.command_init_arg (get_args () .wasm) (
        runtime.arg_required true
    )

### run

In [ ]:
let rec run (matches : runtime.arg_matches) : async.future_pin (resultm.result' u8 resultm.anyhow_error) =
    fun () =>
        inl wasm_path =
            matches
            |> runtime.matches_get_one (get_args () .wasm |> fst)
            |> optionm'.unbox
            |> optionm.value
            |> sm'.from_std_string

        trace Verbose (fun () => "spiral_wasm.run") fun () => { wasm_path }

        inl wasm = wasm_path |> file_system.read |> resultm.try'

        let fn (retry : u8) =
            fun () =>
                inl worker = near_workspaces.sandbox_worker () |> resultm.try'

                trace Verbose (fun () => "spiral_wasm.run") fun () => { retry worker }

                inl contract = worker |> near_workspaces.dev_deploy wasm |> async.await |> resultm.try'

                trace Verbose (fun () => "spiral_wasm.run") fun () => { retry contract }

                inl result =
                    contract
                    |> near_workspaces.call "state_main"
                    |> near_workspaces.transact
                    |> async.await
                    |> resultm.try'

                trace Verbose (fun () => "spiral_wasm.run") fun () => { retry result }

                result
                |> near_workspaces.logs
                |> am'.vec_map sm'.ref_to_std_string
                |> am'.vec_for_each console.write_line

                result |> near_workspaces.print_usd retry

                inl result2 = result |> near_workspaces.into_result

                trace Verbose (fun () => "spiral_wasm.run") fun () => { result2 }

                inl receipt_failures = result |> near_workspaces.receipt_failures
                inl receipt_failures_len = receipt_failures |> am'.vec_len |> i32

                trace Verbose
                    fun () => "spiral_wasm.run"
                    fun () => { receipt_failures_len receipt_failures }

                inl receipt_outcomes = result |> near_workspaces.receipt_outcomes
                inl receipt_outcomes_len = receipt_outcomes |> am'.vec_len |> i32

                trace Verbose
                    fun () => "spiral_wasm.run"
                    fun () => { receipt_outcomes_len receipt_outcomes }

                inl json = result |> near_workspaces.json

                trace Verbose (fun () => "spiral_wasm.run") fun () => { json }

                inl borsh = result |> near_workspaces.borsh

                trace Verbose (fun () => "spiral_wasm.run") fun () => { borsh }

                inl error = { receipt_failures receipt_outcomes_len retry } |> sm'.format
                if receipt_failures_len > 0
                then (Ok (Some error) : _ _ resultm.anyhow_error) |> resultm.box
                elif receipt_outcomes_len > 1
                then (Ok None : _ _ resultm.anyhow_error) |> resultm.box
                else error |> resultm.anyhow_error |> resultm.err
            |> async.new_future_move

        inl rec loop (retry : u8) =
            inl max = 30
            inl init () =
                fun () =>
                    retry
                |> async.new_future_move
            if retry >= max then
                fun () =>
                    init ()
                    |> async.await
                    |> Error
                |> async.new_future_move
            else
                inl result =
                    fn retry
                    |> async.await
                    |> resultm.map_error' sm'.format'
                    |> resultm.unbox
                match result with
                | Ok (None) =>
                    fun () =>
                        init ()
                        |> async.await
                        |> Ok
                    |> async.new_future_move
                | Ok (Some error) =>
                    trace Critical (fun () => "spiral_wasm.run / Ok (Some error)") fun () => { retry error }
                    fun () =>
                        init ()
                        |> async.await
                        |> Error
                    |> async.new_future_move
                | Error error =>
                    trace Warning (fun () => "spiral_wasm.run / Error error") fun () => { retry error }
                    loop (retry + 1)
        inl retries =
            loop 1
            |> async.await

        trace Verbose (fun () => "spiral_wasm.run") fun () => { retries }

        match retries with
        | Ok retries => Ok retries |> resultm.box
        | Error retries => { retries } |> sm'.format |> resultm.anyhow_error |> resultm.err

    |> async.new_future_move

### main

In [ ]:
///! _

inl main (args : array_base string) =
    inl command = get_command ()
    inl arg_matches = command |> runtime.command_get_matches

    inl trace_level =
        arg_matches
        |> runtime.matches_get_one (get_args () .trace_level |> fst)
        |> optionm'.unbox
        |> optionm.map (
            sm'.from_std_string
            >> reflection.union_try_pick
        )
        |> optionm'.flatten
        |> optionm'.default_value Verbose

    inl trace_state = get_trace_state_or_init (Some trace_level)

    trace Verbose
        fun () => $'$"spiral_wasm.main"'
        fun () => { args }

    inl exception : bool =
        arg_matches
        |> runtime.matches_get_flag (get_args () .exception |> fst)

    inl result =
        arg_matches
        |> run
        |> async.block_on
        |> resultm.map_error' sm'.format'

    match result |> resultm.ok' |> optionm'.unbox with
    | Some retries when exception => "spiral_wasm.main / exception=true" |> resultm.err |> resultm.unwrap'
    | None when exception => ()
    | Some retries => retries |> ignore
    | None => result |> resultm.unwrap' |> ignore

    0i32

inl main () =
    $'let main args = !main args' : ()